In [1]:


import numpy as np
import tensorflow as tf

from drowsiness_detection import config
from drowsiness_detection.models import build_dummy_tf_classifier, ThreeDStandardScaler,build_dense_model, build_lstm_model, build_cnn_model


In [2]:
window_size = 10
recording_frequency = 30
exclude_by = "a"

config.set_paths(recording_frequency, window_size)
config.PATHS.WINDOW_DATA

PosixPath('/home/tim/Windows/Windows_30_Hz/WindowData/10_sec')

In [3]:
num_samples = 20
X = np.random.random(num_samples * 300 * 23).reshape((num_samples, 300, 23))
y = np.concatenate((np.zeros((num_samples // 2)), np.ones((num_samples // 2))))
print(X.shape, y.shape)

(20, 300, 23) (20,)


In [4]:
scaler = ThreeDStandardScaler()
X_scaled = scaler.fit_transform(X, y)
print(X_scaled.shape)

(20, 300, 23)


In [5]:
from collections import namedtuple

Input = namedtuple("Input", "X y")
input = Input(X_scaled, y)

In [18]:
def test_model(model, input: Input = input):
    batch_size = 2
    epochs = 2
    X = tf.convert_to_tensor(input.X, dtype=tf.float64)
    y = tf.convert_to_tensor(input.y, dtype=tf.int8)
    model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    score = model.evaluate(X_scaled, y)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

In [12]:
test_model(model=build_dummy_tf_classifier(input_shape=X.shape))

Activation was set to None which is not supported.
Epoch 1/2
9/9 [==============================] - 0s 15ms/step - loss: 5.6489 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 2/2
1/1 [==============================] - 0s 93ms/step - loss: 5.8553 - accuracy: 0.5000
Test loss: 5.855287551879883
Test accuracy: 0.5


In [8]:
test_model(model=build_dense_model(input_shape=X.shape))

Epoch 1/2
9/9 [==============================] - 1s 22ms/step - loss: 0.7101 - accuracy: 0.6667 - val_loss: 0.2701 - val_accuracy: 1.0000
Epoch 2/2
1/1 [==============================] - 0s 97ms/step - loss: 0.0306 - accuracy: 1.0000
Test loss: 0.0306137353181839
Test accuracy: 1.0


In [13]:
test_model(model=build_cnn_model(input_shape=X.shape))

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 300, 23)]         0         
                                                                 
 reshape (Reshape)           (None, 300, 23, 1)        0         
                                                                 
 conv2d (Conv2D)             (None, 99, 23, 32)        192       
                                                                 
 batch_normalization (BatchN  (None, 99, 23, 32)       128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 99, 23, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 95, 23, 32)        5152      
                                                           

In [14]:
test_model(model=build_lstm_model(input_shape=X.shape))

Epoch 1/2
9/9 [==============================] - 4s 251ms/step - loss: 0.7312 - accuracy: 0.3889 - val_loss: 0.8542 - val_accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - 1s 842ms/step - loss: 0.2074 - accuracy: 0.9000
Test loss: 0.20741109549999237
Test accuracy: 0.8999999761581421
